# **Import Data**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers import Adam, Nadam, Adamax

In [2]:
data = pd.read_csv('Breast_Cancer.csv')
data = pd.DataFrame(data)
data

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019,62,Other,Married,T1,N1,IIA,Moderately differentiated,2,Regional,9,Positive,Positive,1,1,49,Alive
4020,56,White,Divorced,T2,N2,IIIA,Moderately differentiated,2,Regional,46,Positive,Positive,14,8,69,Alive
4021,68,White,Married,T2,N1,IIB,Moderately differentiated,2,Regional,22,Positive,Negative,11,3,69,Alive
4022,58,Black,Divorced,T2,N1,IIB,Moderately differentiated,2,Regional,44,Positive,Positive,11,1,72,Alive


# **Klasifikasi Model RNN**

## **Persiapan Data Klasifikasi**

In [3]:
classification_data = data.copy()
classification_data['Status'] = classification_data['Status'].apply(lambda x: 1 if x == 'Alive' else 0)

classification_data

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,1
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,1
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,1
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,1
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019,62,Other,Married,T1,N1,IIA,Moderately differentiated,2,Regional,9,Positive,Positive,1,1,49,1
4020,56,White,Divorced,T2,N2,IIIA,Moderately differentiated,2,Regional,46,Positive,Positive,14,8,69,1
4021,68,White,Married,T2,N1,IIB,Moderately differentiated,2,Regional,22,Positive,Negative,11,3,69,1
4022,58,Black,Divorced,T2,N1,IIB,Moderately differentiated,2,Regional,44,Positive,Positive,11,1,72,1


In [4]:
count_1 = (classification_data == 1).sum().sum()
count_0 = (classification_data == 0).sum().sum()
total_count = count_1 + count_0

print("Persebaran data persentase keselamatan")
print(f"Alive : ", count_1/total_count*100,"%")
print(f"Dead : ", count_0/total_count*100,"%")

Persebaran data persentase keselamatan
Alive :  89.0992744646965 %
Dead :  10.900725535303486 %


## Normalisasi data dengan one hot encoding

In [5]:
# Identify columns with string values
string_columns = classification_data.select_dtypes(include=['object']).columns

# Apply one-hot encoding to string columns
classification_data = pd.get_dummies(classification_data, columns=string_columns)
classification_data = classification_data.replace({True: 1, False: 0})
classification_data

,Age,Tumor Size,Regional Node Examined,Reginol Node Positive,Survival Months,Status,Race_Black,Race_Other,Race_White,Marital Status_Divorced,...,Grade_ anaplastic; Grade IV,Grade_1,Grade_2,Grade_3,A Stage_Distant,A Stage_Regional,Estrogen Status_Negative,Estrogen Status_Positive,Progesterone Status_Negative,Progesterone Status_Positive
0,68,4,24,1,60,1,0,0,1,0,...,0,0,0,1,0,1,0,1,0,1
1,50,35,14,5,62,1,0,0,1,0,...,0,0,1,0,0,1,0,1,0,1
2,58,63,14,7,75,1,0,0,1,1,...,0,0,1,0,0,1,0,1,0,1
3,58,18,2,1,84,1,0,0,1,0,...,0,0,0,1,0,1,0,1,0,1
4,47,41,3,1,50,1,0,0,1,0,...,0,0,0,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019,62,9,1,1,49,1,0,1,0,0,...,0,0,1,0,0,1,0,1,0,1
4020,56,46,14,8,69,1,0,0,1,1,...,0,0,1,0,0,1,0,1,0,1
4021,68,22,11,3,69,1,0,0,1,0,...,0,0,1,0,0,1,0,1,1,0
4022,58,44,11,1,72,1,1,0,0,1,...,0,0,1,0,0,1,0,1,0,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Age                     4024 non-null   int64 
 1   Race                    4024 non-null   object
 2   Marital Status          4024 non-null   object
 3   T Stage                 4024 non-null   object
 4   N Stage                 4024 non-null   object
 5   6th Stage               4024 non-null   object
 6   differentiate           4024 non-null   object
 7   Grade                   4024 non-null   object
 8   A Stage                 4024 non-null   object
 9   Tumor Size              4024 non-null   int64 
 10  Estrogen Status         4024 non-null   object
 11  Progesterone Status     4024 non-null   object
 12  Regional Node Examined  4024 non-null   int64 
 13  Reginol Node Positive   4024 non-null   int64 
 14  Survival Months         4024 non-null   int64 
 15  Stat

# Prediksi Tingkat Keselamatan

## RNN - Splitdata 40-60 - ADAM

In [7]:
# Prepare the data
X = classification_data.drop('Status', axis=1).values  # Drop 'Status' column
y = classification_data[['Status']].values  # Keep only 'Status' column

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Reshape the input data for RNN
X_train = X_train.reshape(-1, 1, len(classification_data.columns)-1)
X_test = X_test.reshape(-1, 1, len(classification_data.columns)-1)

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(1, len(classification_data.columns)-1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='softmax'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=64, validation_data=(X_test, y_test), verbose=0)

# Prediction model
Y_pred = model.predict(X_test)
Y_pred = (Y_pred >= 0.5).astype(int)

# Evaluation metrics
prec = precision_score(y_test, Y_pred, average='weighted')
rec = recall_score(y_test, Y_pred, average='weighted')
acc = accuracy_score(y_test, Y_pred)
f1 = f1_score(y_test, Y_pred, average='weighted')




51/51 [==============================] - 0s 3ms/step


In [8]:
# Print the results
print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")

Precision: 0.8830364916091952
Recall: 0.8919254658385093
Accuracy: 0.8919254658385093
F1 Score: 0.8852879188354907


In [9]:
print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.68      0.51      0.58       238
           1       0.92      0.96      0.94      1372

    accuracy                           0.89      1610
   macro avg       0.80      0.73      0.76      1610
weighted avg       0.88      0.89      0.89      1610



## RNN - Splitdata 30 - 70 - ADAM

In [10]:

# Prepare the data
X = classification_data.drop('Status', axis=1).values  # Drop 'Status' column
y = classification_data[['Status']].values  # Keep only 'Status' column

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Reshape the input data for RNN
X_train = X_train.reshape(-1, 1, len(classification_data.columns)-1)
X_test = X_test.reshape(-1, 1, len(classification_data.columns)-1)

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(1, len(classification_data.columns)-1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='softmax'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=64, validation_data=(X_test, y_test), verbose=0)

# Prediction model
Y_pred = model.predict(X_test)
Y_pred = (Y_pred >= 0.5).astype(int)

# Evaluation metrics
prec = precision_score(y_test, Y_pred, average='weighted')
rec = recall_score(y_test, Y_pred, average='weighted')
acc = accuracy_score(y_test, Y_pred)
f1 = f1_score(y_test, Y_pred, average='weighted')

38/38 [==============================] - 0s 3ms/step


In [11]:
# Print the results
print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")

Precision: 0.8755028151672364
Recall: 0.8882450331125827
Accuracy: 0.8882450331125827
F1 Score: 0.8749631277872086


In [12]:
print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.70      0.40      0.51       175
           1       0.91      0.97      0.94      1033

    accuracy                           0.89      1208
   macro avg       0.80      0.69      0.72      1208
weighted avg       0.88      0.89      0.87      1208



## RNN - Splitdata 20 - 80 - ADAM

In [13]:
# Prepare the data
X = classification_data.drop('Status', axis=1).values  # Drop 'Status' column
y = classification_data[['Status']].values  # Keep only 'Status' column

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data for RNN
X_train = X_train.reshape(-1, 1, len(classification_data.columns)-1)
X_test = X_test.reshape(-1, 1, len(classification_data.columns)-1)

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(1, len(classification_data.columns)-1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='softmax'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=64, validation_data=(X_test, y_test), verbose=0)

# Prediction model
Y_pred = model.predict(X_test)
Y_pred = (Y_pred >= 0.5).astype(int)

# Evaluation metrics
prec = precision_score(y_test, Y_pred, average='weighted')
rec = recall_score(y_test, Y_pred, average='weighted')
acc = accuracy_score(y_test, Y_pred)
f1 = f1_score(y_test, Y_pred, average='weighted')

26/26 [==============================] - 0s 3ms/step


In [14]:
# Print the results
print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")

Precision: 0.8913591008243611
Recall: 0.8981366459627329
Accuracy: 0.8981366459627329
F1 Score: 0.8933544432265661


In [15]:
print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.70      0.56      0.62       120
           1       0.93      0.96      0.94       685

    accuracy                           0.90       805
   macro avg       0.81      0.76      0.78       805
weighted avg       0.89      0.90      0.89       805



## RNN - Splitdata 40-60 - NADAM

In [16]:
# Prepare the data
X = classification_data.drop('Status', axis=1).values  # Drop 'Status' column
y = classification_data[['Status']].values  # Keep only 'Status' column

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Reshape the input data for RNN
X_train = X_train.reshape(-1, 1, len(classification_data.columns)-1)
X_test = X_test.reshape(-1, 1, len(classification_data.columns)-1)

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(1, len(classification_data.columns)-1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='softmax'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Nadam(), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=64, validation_data=(X_test, y_test), verbose=0)

# Prediction model
Y_pred = model.predict(X_test)
Y_pred = (Y_pred >= 0.5).astype(int)

# Evaluation metrics
prec = precision_score(y_test, Y_pred, average='weighted')
rec = recall_score(y_test, Y_pred, average='weighted')
acc = accuracy_score(y_test, Y_pred)
f1 = f1_score(y_test, Y_pred, average='weighted')

51/51 [==============================] - 0s 3ms/step


In [17]:
# Print the results
print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")

Precision: 0.8759572591489423
Recall: 0.8875776397515528
Accuracy: 0.8875776397515528
F1 Score: 0.8773564132910984


In [18]:
print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.68      0.45      0.54       238
           1       0.91      0.96      0.94      1372

    accuracy                           0.89      1610
   macro avg       0.80      0.70      0.74      1610
weighted avg       0.88      0.89      0.88      1610



## RNN - Splitdata 30-70 - NADAM

In [19]:
# Prepare the data
X = classification_data.drop('Status', axis=1).values  # Drop 'Status' column
y = classification_data[['Status']].values  # Keep only 'Status' column

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Reshape the input data for RNN
X_train = X_train.reshape(-1, 1, len(classification_data.columns)-1)
X_test = X_test.reshape(-1, 1, len(classification_data.columns)-1)

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(1, len(classification_data.columns)-1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='softmax'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Nadam(), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=64, validation_data=(X_test, y_test), verbose=0)

# Prediction model
Y_pred = model.predict(X_test)
Y_pred = (Y_pred >= 0.5).astype(int)

# Evaluation metrics
prec = precision_score(y_test, Y_pred, average='weighted')
rec = recall_score(y_test, Y_pred, average='weighted')
acc = accuracy_score(y_test, Y_pred)
f1 = f1_score(y_test, Y_pred, average='weighted')

38/38 [==============================] - 0s 3ms/step


In [20]:
# Print the results
print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")

Precision: 0.8821288903137104
Recall: 0.8923841059602649
Accuracy: 0.8923841059602649
F1 Score: 0.8839383710838166


In [21]:
print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.69      0.47      0.56       175
           1       0.92      0.96      0.94      1033

    accuracy                           0.89      1208
   macro avg       0.80      0.72      0.75      1208
weighted avg       0.88      0.89      0.88      1208



## RNN - Splitdata 20-80 - NADAM

In [22]:
# Prepare the data
X = classification_data.drop('Status', axis=1).values  # Drop 'Status' column
y = classification_data[['Status']].values  # Keep only 'Status' column

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data for RNN
X_train = X_train.reshape(-1, 1, len(classification_data.columns)-1)
X_test = X_test.reshape(-1, 1, len(classification_data.columns)-1)

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(1, len(classification_data.columns)-1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='softmax'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Nadam(), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=64, validation_data=(X_test, y_test), verbose=0)

# Prediction model
Y_pred = model.predict(X_test)
Y_pred = (Y_pred >= 0.5).astype(int)

# Evaluation metrics
prec = precision_score(y_test, Y_pred, average='weighted')
rec = recall_score(y_test, Y_pred, average='weighted')
acc = accuracy_score(y_test, Y_pred)
f1 = f1_score(y_test, Y_pred, average='weighted')

26/26 [==============================] - 0s 2ms/step


In [23]:
# Print the results
print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")

Precision: 0.8877932921543001
Recall: 0.8968944099378882
Accuracy: 0.8968944099378882
F1 Score: 0.8854490119081251


In [24]:
print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.76      0.45      0.57       120
           1       0.91      0.98      0.94       685

    accuracy                           0.90       805
   macro avg       0.84      0.71      0.75       805
weighted avg       0.89      0.90      0.89       805



## RNN - Splitdata 40-60 - ADAMAX


In [25]:
# Prepare the data
X = classification_data.drop('Status', axis=1).values  # Drop 'Status' column
y = classification_data[['Status']].values  # Keep only 'Status' column

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Reshape the input data for RNN
X_train = X_train.reshape(-1, 1, len(classification_data.columns)-1)
X_test = X_test.reshape(-1, 1, len(classification_data.columns)-1)

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(1, len(classification_data.columns)-1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='softmax'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adamax(), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=64, validation_data=(X_test, y_test), verbose=0)

# Prediction model
Y_pred = model.predict(X_test)
Y_pred = (Y_pred >= 0.5).astype(int)

# Evaluation metrics
prec = precision_score(y_test, Y_pred, average='weighted')
rec = recall_score(y_test, Y_pred, average='weighted')
acc = accuracy_score(y_test, Y_pred)
f1 = f1_score(y_test, Y_pred, average='weighted')

51/51 [==============================] - 0s 3ms/step


In [26]:
# Print the results
print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")

Precision: 0.8866460334914079
Recall: 0.8962732919254658
Accuracy: 0.8962732919254658
F1 Score: 0.8856794306069667


In [27]:
print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.74      0.46      0.57       238
           1       0.91      0.97      0.94      1372

    accuracy                           0.90      1610
   macro avg       0.83      0.72      0.75      1610
weighted avg       0.89      0.90      0.89      1610



## RNN - Splitdata 30-70 - ADAMAX

In [28]:
# Prepare the data
X = classification_data.drop('Status', axis=1).values  # Drop 'Status' column
y = classification_data[['Status']].values  # Keep only 'Status' column

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Reshape the input data for RNN
X_train = X_train.reshape(-1, 1, len(classification_data.columns)-1)
X_test = X_test.reshape(-1, 1, len(classification_data.columns)-1)

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(1, len(classification_data.columns)-1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='softmax'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adamax(), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=64, validation_data=(X_test, y_test), verbose=0)

# Prediction model
Y_pred = model.predict(X_test)
Y_pred = (Y_pred >= 0.5).astype(int)

# Evaluation metrics
prec = precision_score(y_test, Y_pred, average='weighted')
rec = recall_score(y_test, Y_pred, average='weighted')
acc = accuracy_score(y_test, Y_pred)
f1 = f1_score(y_test, Y_pred, average='weighted')

38/38 [==============================] - 0s 3ms/step


In [29]:
# Print the results
print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")

Precision: 0.8887755004947857
Recall: 0.8981788079470199
Accuracy: 0.8981788079470199
F1 Score: 0.8892702460652453


In [30]:
print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.72      0.48      0.58       175
           1       0.92      0.97      0.94      1033

    accuracy                           0.90      1208
   macro avg       0.82      0.72      0.76      1208
weighted avg       0.89      0.90      0.89      1208



## RNN - Splitdata 20-80 - ADAMAX


In [31]:
# Prepare the data
X = classification_data.drop('Status', axis=1).values  # Drop 'Status' column
y = classification_data[['Status']].values  # Keep only 'Status' column

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data for RNN
X_train = X_train.reshape(-1, 1, len(classification_data.columns)-1)
X_test = X_test.reshape(-1, 1, len(classification_data.columns)-1)

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(1, len(classification_data.columns)-1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='softmax'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adamax(), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=64, validation_data=(X_test, y_test), verbose=0)

# Prediction model
Y_pred = model.predict(X_test)
Y_pred = (Y_pred >= 0.5).astype(int)

# Evaluation metrics
prec = precision_score(y_test, Y_pred, average='weighted')
rec = recall_score(y_test, Y_pred, average='weighted')
acc = accuracy_score(y_test, Y_pred)
f1 = f1_score(y_test, Y_pred, average='weighted')

26/26 [==============================] - 0s 3ms/step


In [32]:
# Print the results
print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")

Precision: 0.8876825542507691
Recall: 0.8968944099378882
Accuracy: 0.8968944099378882
F1 Score: 0.8860500397895174


In [33]:
print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.75      0.46      0.57       120
           1       0.91      0.97      0.94       685

    accuracy                           0.90       805
   macro avg       0.83      0.72      0.76       805
weighted avg       0.89      0.90      0.89       805

